## Cyberpunks. Urbit Owners

In [1]:
from IPython.core.display import display, HTML

from config import ERC721_ANALYSIS_DATASET_NAME, AZIMUTH_POINTS_TABLE_NAME, SNAPSHOT_TS
from src.utils_bigquery import get_df, create_table_from_df
from src.extractor_urbit import extract_point_data

SELECT_TRANSFERRED_TOKENS = True
DROP_TABLES = True
CREATE_TABLES = True

AZIMUTH_CONTRACT_ADDRESS = '0x223c067F8CF28ae173EE5CafEa60cA44C335fecB'.lower()
LINEAR_STAR_RELEASE_CONTRACT_ADDRESS= '0x86cd9cd0992F04231751E3761De45cEceA5d1801'.lower()
SPAWNED_EVENT_TOPIC = '0xb2d3a6e7a339f5c8ff96265e2f03a010a8541070f3744a247090964415081546'
ZERO_ADDRESS = '0x0000000000000000000000000000000000000000'
AZIMUTH_CONTRACT_CREATE_TS = '2018-11-27 00:00:00'

### Get List of Transferred Tokens

In [2]:
if SELECT_TRANSFERRED_TOKENS:
    query_1 = f'''
    SELECT cast(topics[ORDINAL(3)] AS INT64) as point
    FROM `bigquery-public-data.crypto_ethereum.logs`
    WHERE block_timestamp > '{AZIMUTH_CONTRACT_CREATE_TS}'
      AND block_timestamp < '{SNAPSHOT_TS}'
      AND address = '{AZIMUTH_CONTRACT_ADDRESS}'
      AND topics[ORDINAL(1)] = '{SPAWNED_EVENT_TOPIC}'
    '''

    azimuth_tokens_list = list(get_df(query=query_1).point)
    print(f'Number of Azimuth Tokens: {len(azimuth_tokens_list):>,}')

Number of Azimuth Tokens: 77,050


### Get Point Owners and Types

In [3]:
points_df = extract_point_data(point_list=azimuth_tokens_list, pool_size=10, chunk_size=128)

Processing 77050 points split into 602 chunks on 10 processes:
Points 55753:43346 extracted
Points 106693888:103220480 extracted
Points 4680:25674 extracted
Points 43802:28188 extracted
Points 44631:64621 extracted
Points 61960:35639 extracted
Points 60492:36566 extracted
Points 39584:16050 extracted
Points 35900:52286 extracted
Points 3103392162:1383605414 extracted
Points 1008932774:85984512 extracted
Points 43602:47442 extracted
Points 21322:31261696 extracted
Points 31279:3796 extracted
Points 64877:2985855442 extracted
Points 16305:47266 extracted
Points 28444:42534 extracted
Points 35895:30217 extracted
Points 53822:24894 extracted
Points 328742100:42231 extracted
Points 87098624:85656832 extracted
Points 14591:63826 extracted
Points 31327232:45152 extracted
Points 56245:57521 extracted
Points 3679056310:1592992 extracted
Points 3798:28426 extractedPoints 56349:6693 extracted

Points 42487:58359 extracted
Points 30473:39945 extracted
Points 25662:7714 extracted
Points 45408:8032 

In [4]:
display(HTML(points_df.head().to_html(index=False, notebook=True, show_dimensions=False)))

point,point_size,point_type,owner,parent_point
106693888,2,planet,0xeb5041edcd9b64c12a4f5714de2735276b77aa4e,1280
4170450735,2,planet,0xdc7bb203950dd469a738ef346001f1ad996b4a5b,1839
107218176,2,planet,0xc4d0bf450a8eaebdade66302b6428ab92463019d,1280
107283712,2,planet,0xb0aabc70cb2dac74ae197a8d00776ee9184013a1,1280
107939072,2,planet,0x843ec875de69c0cbdc115610246537b99d8688e4,1280


### Insert Point Data

In [5]:
create_table_from_df(source_df=points_df,
                     dataset_name=ERC721_ANALYSIS_DATASET_NAME,
                     table_name=AZIMUTH_POINTS_TABLE_NAME,
                     drop_existing_table=DROP_TABLES)

Table erc721_analysis:azimuth_points has been deleted.
Table erc721_analysis:azimuth_points has been created.


True